In [9]:
#Dependencies
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle


In [10]:
data_csv = pd.read_csv("student-mat.csv", sep=";")
data_df = pd.DataFrame(data_csv)
data_df
#LEGEND:
#M/Fedu - Education for parents: 0-none, 1-primary, 2-grade 5 to 9, 3-secondary edu, 4-higher edu
#abscenses - total abscenses: 0-93
#studytime - weekly study time: 1-<2 hours, 2-2to5hr, 3-5to10hr, 4->10hr
#internet - Internet access at home Y/N = 0/1
#activities - extracurricular activities Y/N = 0/1
#Pstatis - living together or apart T/A = 0/1
#romantic - in a relationship Y/N = 0/1
#paid - extra paid classes related to course Y/N = 0/1
#famrel - Family Relation Status 1(very bad)-5(excellent) 
#sex - M/F = 0/1

,"school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3"
0,"GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mot..."
1,"GP,F,17,U,GT3,T,1,1,at_home,other,course,fathe..."
2,"GP,F,15,U,LE3,T,1,1,at_home,other,other,mother..."
3,"GP,F,15,U,GT3,T,4,2,health,services,home,mothe..."
4,"GP,F,16,U,GT3,T,3,3,other,other,home,father,1,..."
...,...
390,"MS,M,20,U,LE3,A,2,2,services,services,course,o..."
391,"MS,M,17,U,LE3,T,3,1,services,services,course,m..."
392,"MS,M,21,R,GT3,T,1,1,other,other,course,other,1..."
393,"MS,M,18,R,LE3,T,3,2,services,other,course,moth..."


In [3]:
#Create new dataframe, keeping only the columns we are interested in
data = data_df[["G1","G2","G3","sex", "Medu", "Fedu", "absences", "internet", "studytime", "failures", "activities", "Pstatus", "romantic", "paid","famrel"]]
data1 = pd.DataFrame(data)
#Replacing str values with 0 and 1 to make it readable for the linear regression model
data1.replace('F',1,inplace=True)
data1.replace('M',0,inplace=True)
data1.replace('yes',0,inplace=True)
data1.replace('no',1,inplace=True)
data1.replace('A',1,inplace=True)
data1.replace('T',0,inplace=True)
data2 = data.copy()
data_binary = pd.get_dummies(data2, columns=["sex", "internet", "activities", "Pstatus", "romantic", "paid"])
pd.set_option('max_columns', None)
data_binary.head()

KeyError: "None of [Index(['G1', 'G2', 'G3', 'sex', 'Medu', 'Fedu', 'absences', 'internet',\n       'studytime', 'failures', 'activities', 'Pstatus', 'romantic', 'paid',\n       'famrel'],\n      dtype='object')] are in the [columns]"

In [ ]:
data_binary.rename(columns={"sex_1": "sex_f", "sex_M": "sex_m"})


In [ ]:
#Linear Regression model
predict = "G3"
x = np.array(data_binary.drop([predict], 1))
y = np.array(data_binary[predict])
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = .2, random_state=8)
linear = linear_model.LinearRegression()
linear.fit(x_train, y_train)
acc = linear.score(x_test, y_test)
print(acc)
print(linear.coef_)
print(linear.intercept_)

In [ ]:
predictions = linear.predict(x_test)
for x in range(len(predictions)):
    print(predictions[x], x_test[x], y_test[x])
#How to read: EV[G1, G2, sex, Medu, Fedu, abscences, internet, studytime, failures, activities, Pstatus, romantic, paid, famrel]AV
#EV - Expected Value, AV - Actual Value

In [ ]:
style.use("ggplot")